In [1]:
from dotenv import load_dotenv
import os
import base64
from requests import post, get
import json


load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

def getToken():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    data = {"grant_type": "client_credentials"}

    result = post(url, headers=headers, data=data)
    json_results = json.loads(result.content)
    token = json_results["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = "?q={0}&type=artist&limit=1".format(artist_name)
    query_url = url + query
    result = get(query_url, headers=headers)
    json_result = json.loads(result.content)["artists"]["items"]
    if len(json_result) == 0:
        print("No artist with this name exists")
    
    return json_result[0]

def get_songs_by_artist(token, artist_id):
    url = "https://api.spotify.com/v1/artists/{}/top-tracks?country=US".format(artist_id)
    print(url)
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)["tracks"]
    return json_result

def get_songs_from_playlist(token, playlist_id):
    url = "https://api.spotify.com/v1/playlists/{}/tracks".format(playlist_id)
    headers = get_auth_header(token)
    result = get(url, headers=headers)
    json_result = json.loads(result.content)["items"]

    for idx, song in enumerate(json_result):
        print("{0}. {1}".format(idx+1, song["track"]["name"]))


In [27]:
import csv
import pandas as pd
import matplotlib as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv("datasets/dataset.csv")


# COLUMNS:
# ['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name',
#        'popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
#        'key', 'loudness', 'mode', 'speechiness', 'acousticness',
#        'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
#        'track_genre']

df = df.dropna()
df.drop(columns="Unnamed: 0", inplace=True)
df = df.drop_duplicates('track_id')

numerical_features = ['explicit', 'popularity', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence']

scaled_df = df
scaler = MinMaxScaler()
scaled_df[numerical_features] = scaler.fit_transform(df[numerical_features])



In [32]:
def generate_recommended_songs(track_name, num_recommended):
    if track_name not in scaled_df["track_name"].values:
        print("Song doesn't exist")
        return

    track_index = scaled_df.index[scaled_df["track_name"] == track_name]

    scores = cosine_similarity(scaled_df.loc[track_index][numerical_features], scaled_df[numerical_features])

    similar_song_indices = scores.argsort()[0][::-1][1:num_recommended + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = scaled_df.iloc[similar_song_indices][["track_name", "artists"]]
    print(content_based_recommendations)



print(generate_recommended_songs("", 5))


Song doesn't exist
None
